# [모의 경진대회] 뇌파 데이터를 이용한 수면 단계 분류

* 수치/시계열 분류 과제
* 담당: 박성호M

## 데이터 디렉토리 구조

In [93]:
01_DATA/  
  \_train/  
    \_xxx.npy  
    \_yyy.npy
    \_zzz.npy
    \_...  
  \_test/  
    \_aaa.npy  
    \_bbb.npy  
    \_...  
  \_train_labels.csv  
  \_sample_submission.csv

SyntaxError: invalid decimal literal (2388783385.py, line 1)

## 필수 라이브러리 불러오기

In [1]:
import os
import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

## 하이퍼파라미터 및 기타 인자 설정

#### 데이터 경로

In [2]:
# 데이터 디렉토리 및 클래스 개수 
DATA_DIR = '/workspace/Competition/PSG/01_DATA/trial3'
label_dir = os.path.join(DATA_DIR, 'train_labels.csv')
train_dir = os.path.join(DATA_DIR, 'train')
test_dir = os.path.join(DATA_DIR, 'test')
norm_dir = os.path.join(DATA_DIR, 'norm.npy')

#### 시드 설정

In [3]:
RANDOM_SEED = 2022

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

#### 디바이스 설정

In [4]:
# 디바이스 설정 (GPU 설정)
os.environ["CUDA_VISIBLE_DEVICES"]="0"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#### 하이퍼파라미터 설정

In [5]:
EPOCHS = 20
BATCH_SIZE = 32
LEARNING_RATE = 0.0003
EARLY_STOPPING_PATIENCE = 10

## Dataset 정의

In [6]:
class EEG_Single_Dataset(Dataset):
    def __init__(self, datapath, labeldf, normpath):
        self.df = labeldf
        self.label_encoding = {'W':0, 'N1':1, 'N2':2, 'N3':3, 'R':4}
        self.data_path = datapath
        self.file_ids = self.df['rec_id']
        self.labels = self.df['stage']
        self.normparams = np.load(normpath).astype('float32')
        self.mean = self.normparams[0]
        self.std = self.normparams[1]
    
    def __len__(self):
        return len(self.file_ids)
    
    def __getitem__(self,index):
        npypath = os.path.join(self.data_path, self.file_ids[index]+'.npy')
        x = torch.from_numpy(np.load(npypath).astype('float32'))
        x = (x-self.mean)/self.std
        subx = x[:,-30*128:]
        label = self.labels[index]
        y = self.label_encoding[label]
        
        return subx,y

## 모델 정의

In [7]:
class DOUBLE_CNN(nn.Module):
    def __init__(self):
        super(DOUBLE_CNN, self).__init__()
        
        self.small_cnn = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=64, kernel_size=int(128/2), stride = int(128/16)),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=8, stride=8),
            nn.Dropout(p=0.3),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=4),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(in_channels=128, out_channels=128, kernel_size=4),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(in_channels=128, out_channels=128, kernel_size=4),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=4),
        )

        self.large_cnn = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=64, kernel_size=128*4, stride=int(128/2)),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=4),
            nn.Dropout(p=0.3),
            nn.Conv1d(in_channels = 64, out_channels=128, kernel_size=3),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(in_channels = 128, out_channels = 128, kernel_size=3),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
        )
        
        self.fc = nn.Sequential(
            nn.Linear((12+4)*128,1024),
            nn.ReLU(),
            nn.Linear(1024,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,5)
        )

    def forward(self, x):
        xs = self.small_cnn(x)
        xl = self.large_cnn(x)
        xs = xs.flatten(1,2)
        xl = xl.flatten(1,2)
        xcat = torch.cat((xs,xl),1)
        out = self.fc(xcat)
        return out

## Utils 정의
#### EarlyStopper

In [8]:
class LossEarlyStopper():
    def __init__(self, patience: int)-> None:
        self.patience = patience
        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False
        self.savel_model = False
        
    def check_early_stopping(self, loss: float)-> None:
        if loss > self.min_loss:
            self.patience_counter +=1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"
            
            if self.patience_counter == self.patience:
                self.stop=True
            
        else:
            self.patience_counter = 0
            self.save_model = True
            msg = f"Validation loss decreased {self.min_loss} - > {loss}"
            self.min_loss = loss
        print(msg)

#### Trainer

In [9]:
class Trainer():
    def __init__(self, model, optimizer, loss, metrics, device):
        self.model = model
        self.optimizer = optimizer
        self.loss = loss
        self.metric_fn = metrics
        self.device = device
        
    def train_epoch(self, dataloader, epoch_index):
        self.model.train()
        train_total_loss = 0
        target_list = []
        pred_list = []
        
        for batch_index, (x,y) in enumerate(dataloader):
            x,y = x.to(self.device), y.to(self.device)
            y_pred = model(x)
            loss = self.loss(y_pred,y)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            train_total_loss += loss.item()
            pred_list.extend(y_pred.argmax(dim=1).cpu().tolist())
            target_list.extend(y.cpu().tolist())
        self.train_mean_loss = train_total_loss / (batch_index+1)
        train_score, f1 = self.metric_fn(y_pred=pred_list, y_answer=target_list)
        msg = f"Epoch {epoch_index}, Train loss: {self.train_mean_loss}, Acc:{train_score}, F1-Macro: {f1}"
        print(msg)
    
    def validate_epoch(self, dataloader, epoch_index):
        val_total_loss = 0
        target_list = []
        pred_list = []
        
        with torch.no_grad():
            for batch_index, (x, y) in enumerate(dataloader):
                x = x.to(self.device)
                y = y.to(self.device)
                y_pred = self.model(x)
                loss = self.loss(y_pred, y)
                
                val_total_loss += loss.item()
                target_list.extend(y.cpu().tolist())
                pred_list.extend(y_pred.argmax(dim=1).cpu().tolist())
        self.val_mean_loss = val_total_loss / (batch_index+1)
        val_score, f1 = self.metric_fn(y_pred = pred_list, y_answer = target_list)
        msg = f"Epoch {epoch_index}, Val loss: {self.val_mean_loss}, Acc:{val_score}, F1-Macro: {f1}"
        print(msg)

#### 평가지표

In [10]:
def get_metric_fn(y_pred, y_answer):
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not the same.'
    accuracy = accuracy_score(y_answer, y_pred)
    f1 = f1_score(y_answer, y_pred, average='macro')
    return accuracy, f1

## 모델 학습

#### Dataset & Dataloader 설정

In [11]:
# Load label dataframe
entiredf = pd.read_csv(label_dir)
traindf, valdf = train_test_split(entiredf, test_size=0.2)
traindf = traindf.reset_index(drop=True)
valdf = valdf.reset_index(drop=True)


train_dataset = EEG_Single_Dataset(datapath=train_dir, labeldf=traindf, normpath=norm_dir)
val_dataset = EEG_Single_Dataset(datapath=train_dir, labeldf=valdf, normpath=norm_dir)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print('Train set samples: ', len(train_dataset), 'Val set samples: ', len(val_dataset))

Train set samples:  20024 Val set samples:  5007


#### 모델과 기타 utils 설정

In [12]:
model = DOUBLE_CNN().to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE)
metrics = get_metric_fn

trainer = Trainer(model, optimizer, loss_fn, get_metric_fn, DEVICE)

In [13]:
model

DOUBLE_CNN(
  (small_cnn): Sequential(
    (0): Conv1d(1, 64, kernel_size=(64,), stride=(8,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=8, stride=8, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.3, inplace=False)
    (5): Conv1d(64, 128, kernel_size=(4,), stride=(1,))
    (6): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU()
    (8): Conv1d(128, 128, kernel_size=(4,), stride=(1,))
    (9): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Conv1d(128, 128, kernel_size=(4,), stride=(1,))
    (12): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU()
    (14): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (large_cnn): Sequential(
    (0): Conv1d(1, 64, kernel_size=(512,), stride=(64,))
    (1): Batc

### Epoch 단위 학습 진행

In [ ]:
for epoch_index in tqdm(range(EPOCHS)):
    trainer.train_epoch(train_loader, epoch_index)
    trainer.validate_epoch(val_loader, epoch_index)
    
    early_stopper.check_early_stopping(loss = trainer.val_mean_loss)
    
    if early_stopper.stop:
        print('Early Stopped')
        break
    if early_stopper.save_model:
        check_point = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
        torch.save(check_point, 'best.pt')

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 0, Train loss: 1.1453338464418539, Acc:0.5292149420695166, F1-Macro: 0.38243259930365275


  5%|▌         | 1/20 [00:52<16:33, 52.28s/it]

Epoch 0, Val loss: 1.0612052914443288, Acc:0.5674056321150389, F1-Macro: 0.4022797071031422
Validation loss decreased inf - > 1.0612052914443288
Epoch 1, Train loss: 1.0100702574839606, Acc:0.5957850579304834, F1-Macro: 0.47062324335276057


 10%|█         | 2/20 [01:16<10:42, 35.71s/it]

...

Epoch 16, Val loss: 0.7536191109828888, Acc:0.7191931296185341, F1-Macro: 0.5839228846198454
Validation loss decreased 0.7560649206683894 - > 0.7536191109828888
Epoch 17, Train loss: 0.6399173000797677, Acc:0.7546444266879744, F1-Macro: 0.6273127578012092


 90%|█████████ | 18/20 [07:26<00:46, 23.36s/it]

Epoch 17, Val loss: 0.7639277095247985, Acc:0.7151987217894947, F1-Macro: 0.5866299300537967
Early stopping counter 1/10
Epoch 18, Train loss: 0.62059688591919, Acc:0.7645825009988014, F1-Macro: 0.6450756993283382


 95%|█████████▌| 19/20 [07:49<00:23, 23.27s/it]

Epoch 18, Val loss: 0.7489563065349676, Acc:0.7269822248851607, F1-Macro: 0.6028545935572144
Validation loss decreased 0.7536191109828888 - > 0.7489563065349676


## 추론

#### 테스트 Dataset 정의

In [199]:
class TestDataset(Dataset):
    def __init__(self, datapath, normpath):
        self.data_path = datapath
        self.npy_list = os.listdir(self.data_path)
        self.normparams = np.load(normpath).astype('float32')
        self.mean = self.normparams[0]
        self.std = self.normparams[1]
    
    def __len__(self):
        return len(self.npy_list)
    
    def __getitem__(self,index):
        filename = self.npy_list[index]
        npypath = os.path.join(self.data_path, filename)
        x = torch.from_numpy(np.load(npypath).astype('float32'))
        x = (x-self.mean)/self.std
        subx = x[:,-30*128:]
        subfilename = filename.split('.')[0]
        return subx,subfilename

#### 테스트 Dataset & Dataloader 설정

In [200]:
test_dataset = TestDataset(test_dir, norm_dir)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

#### 모델 로드

In [201]:
TRAINED_MODEL_PATH = 'best.pt'
test_model = DOUBLE_CNN()
test_model.load_state_dict(torch.load(TRAINED_MODEL_PATH)['model'])

<All keys matched successfully>

#### 추론 진행

In [213]:
file_list = []
pred_list = []

test_model.to(DEVICE)
test_model.eval()
with torch.no_grad():
    for batch_index, (x,y) in tqdm(enumerate(test_loader)):
        x = x.to(DEVICE)
        pred = test_model(x)
        
        file_list.extend(list(y))
        pred_list.extend(pred.argmax(dim=1).tolist())

196it [00:02, 70.27it/s]


#### 결과 저장

In [222]:
# 예측 결과 데이터프레임으로 저장
results = pd.DataFrame({'rec_id':file_list, 'stage':pred_list})

# 예측 숫자 --> 클래스로 변경
label_decoding = {0:'W', 1:'N1', 2:'N2', 3:'N3', 4:'R'}
results = results.replace(label_decoding)

# sample_submission.csv와 순서 동일하게 변경
sampledf = pd.read_csv(os.path.join(DATA_DIR,'sample_submission.csv'))
sorter = list(sampledf['rec_id'])
results = results.set_index('rec_id')
results = results.loc[sorter].reset_index()

# 결과 저장
results.to_csv('prediction.csv',index=False)